# Prerequisites

In [ ]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [2]:
curr = pd.read_csv('today.csv',index_col='Trade/OPS Ref',low_memory=False)
pre = pd.read_csv('yesterday.csv',index_col='Trade/OPS Ref',low_memory=False)
curr = curr[~curr["Trade Type"].isin(['FX_BK'])]
pre = pre[~pre["Trade Type"].isin(['FX_BK'])]


## Dealing with Duplicates in Physical

In [3]:
dupli_list = curr[curr.index.duplicated()].index.unique().tolist()
for i in dupli_list:
    curr.loc[i, 'Quantity'] = curr.loc[i, 'Quantity'].sum()
    curr.loc[i, 'm2m'] = curr.loc[i, 'm2m'].sum()
curr = curr[~curr.index.duplicated(keep='first')]
dupli_list = pre[pre.index.duplicated()].index.unique().tolist()
for i in dupli_list:
    pre.loc[i, 'Quantity'] = pre.loc[i, 'Quantity'].sum()
    pre.loc[i, 'm2m'] = pre.loc[i, 'm2m'].sum()
pre = pre[~pre.index.duplicated(keep='first')]

## Dealing with Date

In [4]:
cols = ['Date', 'Shipment Month', 'Terminal Month', 'M2M Terminal Month']
for i in cols:
    formatted_dates = pd.to_datetime(pre[i], format='mixed', errors='coerce')
    pre[i] = formatted_dates
for i in cols:
    formatted_dates = pd.to_datetime(curr[i], format='mixed', errors='coerce')
    curr[i] = formatted_dates


# Group Based on Units

In [5]:
group = input("Enter the units you want to group (comma separated): ")
group_list = group.split(',')
group = [i.strip() for i in group_list]
print(f"Grouping the following units: {group}")
group_data = curr[curr['Unit'].isin(group)]
group_data_pre = pre[pre['Unit'].isin(group)]
name = input("Enter a name for the group: ")
os.makedirs(f"Group Files/{name}", exist_ok=True)
group_data.to_csv(f"Group Files/{name}/unit_data_today.csv", index=True)
group_data_pre.to_csv(f"Group Files/{name}/unit_data_yesterday.csv", index=True)

Grouping the following units: ['GCB', 'GR-BSC', 'GR-SPAIN']


# Calculating DoD PnL

## Futures PnL

In [6]:
fut_table = pd.DataFrame()
gr_today = pd.read_csv(f"Group Files/{name}/unit_data_today.csv",index_col=0)
gr_yesterday = pd.read_csv(f"Group Files/{name}/unit_data_yesterday.csv",index_col=0)
gr_today,gr_yesterday = gr_today.align(gr_yesterday, join='inner')

In [7]:
gr_today

,Product,Vessel__Strategy,Unit,OPS Unit,ORSUnit,Date,Party,Party Ref,Grade,Shipment Month,...,Broker,Broker Reference,Party Code,Fixing Date,Grade Optionality,Design,Provisional Price,BL Date,BL Quantity,Outport
Trade/OPS Ref,,,,,,,,,,,,,,,,,,,,,
X29239100003,Maize,6494 - GCB Argentina Corn,GCB,GCB,GCB,2025-04-16,J.P. MORGAN SECURITIES PLC A/C 9A367,NaN,NaN,2025-07-01,...,NaN,NaN,J1176,NaN,NaN,0,NaN,NaN,NaN,NaN
X29280600008,Maize,6494 - GCB Argentina Corn,GCB,GCB,GCB,2025-04-30,J.P. MORGAN SECURITIES PLC A/C 9A367,NaN,NaN,2025-07-01,...,NaN,NaN,J1176,NaN,NaN,0,NaN,NaN,NaN,NaN
X29289900055,Maize,6494 - GCB Argentina Corn,GCB,GCB,GCB,2025-05-05,J.P. MORGAN SECURITIES PLC A/C 9A367,NaN,NaN,2025-07-01,...,NaN,NaN,J1176,NaN,NaN,0,NaN,NaN,NaN,NaN
X29289900059,Maize,6494 - GCB Argentina Corn,GCB,GCB,GCB,2025-05-05,J.P. MORGAN SECURITIES PLC A/C 9A367,NaN,NaN,2025-07-01,...,NaN,NaN,J1176,NaN,NaN,0,NaN,NaN,NaN,NaN
X29297900001,Maize,6494 - GCB Argentina Corn,GCB,GCB,GCB,2025-05-07,J.P. MORGAN SECURITIES PLC A/C 9A367,NaN,NaN,2025-07-01,...,NaN,NaN,J1176,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25/S/06040/A,Maize,8014 - BSC Spain,GR-SPAIN,GR-SPAIN,GR-SPAIN,2025-06-19,"OLAM GRAINS SPAIN, S.L.U.",NaN,ROM MZ,2025-11-01,...,DIRECT,NaN,O1541,NaN,NaN,0,NaN,NaN,NaN,N
25/S/06041/A,Maize,8014 - BSC Spain,GR-SPAIN,GR-SPAIN,GR-SPAIN,2025-06-19,"OLAM GRAINS SPAIN, S.L.U.",NaN,ROM MZ,2025-12-01,...,DIRECT,NaN,O1541,NaN,NaN,0,NaN,NaN,NaN,N
5141000015547,Wheat,8014 - BSC Spain,GR-BSC,GR-BSC,GR-BSC,2025-05-30,NaN,NaN,NaN,2025-06-01,...,NaN,NaN,NaN,2025-06-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN


In [8]:
fut_today = gr_today[gr_today['Trade Type'] == 'FUT']
fut_yester = gr_yesterday[gr_yesterday['Trade Type'] == 'FUT']
fut_table['Futures PnL'] = (fut_today['m2m'] - fut_yester['m2m'])
fut_table['M2M Terminal Month'] = fut_today['M2M Terminal Month']
fut_table['Strategy Trader'] = fut_today['Strategy Trader']

In [9]:
fut_table

,Futures PnL,M2M Terminal Month,Strategy Trader
Trade/OPS Ref,,,
X29239100003,47925.0,NaN,GCB-GR
X29280600008,-5400.0,2025-07-01,GCB-GR
X29289900055,-2362.5,2025-07-01,GCB-GR
X29289900059,-13162.5,2025-07-01,GCB-GR
X29297900001,-17887.5,2025-07-01,GCB-GR
...,...,...,...
X29201300007,1600.0,2025-12-01,PEDRO VAS
X29201300008,-1600.0,2025-12-01,PEDRO VAS
X29201300010,1600.0,2025-12-01,PEDRO VAS


## FX PnL

In [10]:
fx_table = pd.DataFrame()

In [11]:
fx_today = gr_today[gr_today['Trade Type'] == 'FX']
fx_yester = gr_yesterday[gr_yesterday['Trade Type'] == 'FX']
fx_table['FX PnL'] = (fx_today['m2m'] - fx_yester['m2m'])
fx_table['Shipment Month'] = fx_today['Shipment Month']
fx_table['Strategy Trader'] = fx_today['Strategy Trader']

## Physical PnL

In [12]:
phy_table = pd.DataFrame()

In [13]:
# PHYICAL Future Conversion factor dict
conversion_dict = {
    'Barley':        round(0.01 * 2204.62 / 48.0,4),     # 48 lb/bu
    'Bran':          round(0.01 * 2204.62 / 20.0,4),     # estimated (bran weight varies)
    'CDSBO':         round(0.01 * 2204.62 / 60.0,4),     # Crude Degummed Soybean Oil ~60 lb/bu equivalent
    'CPL':           round(0.01 * 2204.62 / 60.0,4),     # Crude Palm Olein (estimate)
    'CPO':           round(0.01 * 2204.62 / 60.0,4),     # Crude Palm Oil (estimate)
    'Maize':         round(0.01 * 2204.62 / 56.0,4),     # aka Corn
    'PFAD':          round(0.01 * 2204.62 / 60.0,4),     # Palm Fatty Acid Distillate (estimate)
    'RPL':           round(0.01 * 2204.62 / 60.0,4),     # RBD Palm Olein (estimate)
    'RPO':           round(0.01 * 2204.62 / 60.0,4),     # RBD Palm Oil (estimate)
    'Soya Bean':     round(0.01 * 2204.62 / 60.0,4),     # 60 lb/bu
    'Soyabean Meal': round(0.01 * 2204.62 / 48.0,4),     # estimate
    'Wheat':         round(0.01 * 2204.62 / 60.0,4),     # average across classes
    'RSO':           round(0.01 * 2204.62 / 60.0,4),     # Refined Soybean Oil
    'CSFO':          round(0.01 * 2204.62 / 60.0,4),     # Crude Sunflower Oil (estimate)
    'RAPEMEAL':      round(0.01 * 2204.62 / 50.0,4),     # Rapeseed Meal (estimate)
}


In [14]:
phy_today = gr_today[(gr_today['Transaction Type'] == 'PHY') & (gr_today['Trade Type'] == 'FIXED')]
phy_yester = gr_yesterday[(gr_yesterday['Transaction Type'] == 'PHY') & (gr_yesterday['Trade Type'] == 'FIXED')]

#Future Part
phy_today['USD/MT'] = phy_today.apply(lambda row: row['Market Rate'] *row["Qty Exposure"]* conversion_dict.get(row['Product'], 1), axis=1)
phy_yester['USD/MT'] = phy_yester.apply(lambda row: row['Market Rate'] *row["Qty Exposure"]* conversion_dict.get(row['Product'], 1), axis=1)

phy_table['Future Part'] = phy_today['USD/MT'] - phy_yester['USD/MT']

#FX Part
euro_tod = phy_today[(phy_today['Exch'] == "EOP")&(phy_today['Contract Currency']=='EUR')][0:1]
euro_yes = phy_yester[(phy_today['Exch'] == "EOP")&(phy_yester['Contract Currency']=='EUR')][0:1]
day_change = euro_tod["US$/Ton"]/euro_tod["Price"] - euro_yes["US$/Ton"]/euro_yes["Price"]
phy_table['FX Part'] = (phy_yester['M2M Currency Risk'])*day_change.values[0]

phy_table['Total PHY PnL'] = (phy_today['m2m'] - phy_yester['m2m'])


phy_table.fillna(0, inplace=True)
phy_table["PHY PnL"] =  phy_table['Total PHY PnL']-(phy_table['Future Part'] + phy_table['FX Part'])
phy_table['Strategy Trader'] = phy_today['Strategy Trader']
phy_table["Shipment Month"] = phy_today['Shipment Month']


C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\726142089.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phy_today['USD/MT'] = phy_today.apply(lambda row: row['Market Rate'] *row["Qty Exposure"]* conversion_dict.get(row['Product'], 1), axis=1)
C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\726142089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phy_yester['USD/MT'] = phy_yester.apply(lambda row: row['Market Rate'] *row["Qty Exposure"]* conversion_dict.get(row['Product'], 1), axis=1)
C:\Users

In [15]:
phy_table

,Future Part,FX Part,Total PHY PnL,PHY PnL,Strategy Trader,Shipment Month
Trade/OPS Ref,,,,,,
23/S/05170/A,1745.1500,200.100000,5758.07,3812.820000,GCB-GR,2024-05-01
23/S/05894/A,1047.0900,-44.160000,3290.62,2287.690000,GCB-GR,2024-05-01
23/S/06987/A,188.4762,13.413600,613.68,411.790200,GCB-GR,2024-05-01
23/S/07350/B,872.5750,-64.400000,2714.59,1906.415000,GCB-GR,2024-11-01
23/S/07351/B,872.5750,-64.400000,2714.59,1906.415000,GCB-GR,2024-10-01
...,...,...,...,...,...,...
25/S/91939/A,1058.2656,618.023616,1676.24,-0.049216,PEDRO VAS,2025-12-01
25/S/91941/A,974.4075,554.566800,1528.93,-0.044300,PEDRO VAS,2025-09-01
25/S/91942/A,944.8800,554.566800,1499.40,-0.046800,PEDRO VAS,2025-10-01


# Making Pivot Tables using PnL

In [43]:
future_pivot = fut_table.pivot_table(index="Strategy Trader",columns='M2M Terminal Month', values='Futures PnL', aggfunc='sum')
fx_pivot = fx_table.pivot_table(index='Strategy Trader', columns='Shipment Month', values='FX PnL', aggfunc='sum')
phy_pivot = phy_table.pivot_table(index='Strategy Trader',columns='Shipment Month', values='PHY PnL', aggfunc='sum')

In [44]:
future_pivot.fillna(0, inplace=True)
future_pivot = future_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else np.nan)
future_pivot


C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\3757803474.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  future_pivot = future_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else np.nan)


M2M Terminal Month,2025-07-01,2025-09-01,2025-12-01,2026-03-01,2026-05-01,2026-09-01
Strategy Trader,,,,,,
ALEXANDRES,0,0,0,0,0,0
GCB-GR,"-47,925","81,394",0,0,0,0
PEDRO VAS,"135,675","-276,999","-180,870","-40,800","-25,962",0


In [45]:
fx_pivot.fillna(0, inplace=True)
fx_pivot = fx_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else np.nan)
fx_pivot

C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\1142131879.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fx_pivot = fx_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else np.nan)


Shipment Month,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01
Strategy Trader,,,,,,,
GCB-GR,"1,406,827",0,0,0,0,0,0
PEDRO VAS,"-1,648,702","-248,037","-52,448","-54,203","-52,466","-52,881","-52,199"


## Checking For Physical Change Factors

In [63]:
change_table_today = phy_today[phy_table['PHY PnL'] > 100]
change_table_yester = phy_yester[phy_table['PHY PnL'] > 100]
change_table_today.fillna(0,inplace=True)
change_table_yester.fillna(0,inplace=True)

C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\3710944504.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  change_table_today = phy_today[phy_table['PHY PnL'] > 100]
C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\3710944504.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  change_table_yester = phy_yester[phy_table['PHY PnL'] > 100]
C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\3710944504.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  change_table_today.fillna(0,inplace=True)
C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\3710944504.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

### Dissecting Physical PnL into Factors

In [64]:
common_index = change_table_today.index.intersection(change_table_yester.index)

#### Quantity


In [65]:
phy_change_quantity_bool = change_table_today.loc[common_index]['Quantity'] != change_table_yester.loc[common_index]['Quantity']
phy_change_quantity_today = change_table_today.loc[common_index][phy_change_quantity_bool]
phy_change_quantity_yester = change_table_yester.loc[common_index][phy_change_quantity_bool]

In [66]:
phy_change_quantity_final = phy_change_quantity_today['Quantity'] - phy_change_quantity_yester['Quantity']
phy_change_quantity_final = phy_change_quantity_final.to_frame(name='Quantity Change')
phy_change_quantity_final

,Quantity Change
Trade/OPS Ref,


#### Market Diff

In [67]:
phy_change_mdiff_bool = change_table_today.loc[common_index]['Market Diff'] != change_table_yester.loc[common_index]['Market Diff']
phy_change_mdiff_today = change_table_today.loc[common_index][phy_change_mdiff_bool]
phy_change_mdiff_yester = change_table_yester.loc[common_index][phy_change_mdiff_bool]
phy_change_mdiff_final = phy_change_mdiff_today['Market Diff'] - phy_change_mdiff_yester['Market Diff']
phy_change_mdiff_final = phy_change_mdiff_final.to_frame(name='Market Diff Change')

In [68]:
phy_change_mdiff_final

,Market Diff Change
Trade/OPS Ref,
25/P/91956/A,75.0
25/P/90415/A,4.0
25/P/90462/A,4.0
25/P/90463/A,3.0
25/P/90522/A,4.0
25/S/01749/A,-3.0
25/S/01750/A,-6.0
24/P/92592/A,168.0


#### Trade Price

In [69]:
phy_change_tprice_bool = change_table_today.loc[common_index]['Price'] != change_table_yester.loc[common_index]['Price']
phy_change_tprice_today = change_table_today.loc[common_index][phy_change_tprice_bool]
phy_change_tprice_yester = change_table_yester.loc[common_index][phy_change_tprice_bool]
phy_change_tprice_final = phy_change_tprice_today['Price'] - phy_change_tprice_yester['Price']
phy_change_tprice_final = phy_change_tprice_final.to_frame(name='Trade Price Change')

In [70]:
phy_change_tprice_final

,Trade Price Change
Trade/OPS Ref,


#### Cost

In [71]:
phy_change_cost_bool = change_table_today.loc[common_index]['Cost'] != change_table_yester.loc[common_index]['Cost']
phy_change_cost_today = change_table_today.loc[common_index][phy_change_cost_bool]
phy_change_cost_yester = change_table_yester.loc[common_index][phy_change_cost_bool]
phy_change_cost_final = phy_change_cost_today['Cost'] - phy_change_cost_yester['Cost']
phy_change_cost_final = phy_change_cost_final.to_frame(name='Cost Change')

In [72]:
phy_change_cost_final

,Cost Change
Trade/OPS Ref,


### Exporting as excel Sheet

In [73]:
wb = openpyxl.Workbook()
excel_path = f"Group Files/{name}/DoD_PnL.xlsx"

future_pivot = future_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else 0)
fx_pivot = fx_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else 0)
phy_pivot = phy_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else 0)

C:\Users\harsh\AppData\Local\Temp\ipykernel_18960\2913573856.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  future_pivot = future_pivot.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else 0)


ValueError: Unknown format code 'f' for object of type 'str'